In [6]:
import pandas as pd
from des_helpers import dpa_select_function, encrypt, decrypt, bin2hex, des, preprocess_key, dec2bin

# Load data
trace_df = pd.read_csv("processed_trace_data.csv")

ciphertexts = [c.upper() for c in trace_df['Ciphertext']]
plaintexts = [p.upper() for p in trace_df['Plaintext']]

In [7]:
trace_df.iloc[0]

Filepath      secmatv1_2006_04_0809/wave_DES_HW_2006-04-09_0...
Key                                            6a65786a65786a65
Plaintext                                      c80e9f96b23813ff
Ciphertext                                     8d28b211cf8d2683
Power                                                  0.072367
Name: 0, dtype: object

In [8]:
# Test encrypt decrypt
sample = trace_df.iloc[0]
key = sample["Key"].upper()
pt = sample["Plaintext"].upper()
ct = sample["Ciphertext"].upper()

text = bin2hex(encrypt(pt, key))
print(text == ct)

text = bin2hex(decrypt(ct, key))
print(text == pt)

True
True


In [17]:
# Test select func
def mod_decrypt(ciphertext, key):
	rkb = preprocess_key(key)
	rkb_rev = rkb[::-1]
	text, sbox_out = des(ciphertext, rkb_rev)

	return text, sbox_out

def mod_encrypt(plaintext, key):
	rkb = preprocess_key(key)
	text, sbox_out = des(plaintext, rkb)

	return text, sbox_out

for i in range(10):
	ct = ciphertexts[i]
	pt = plaintexts[i]

	text, sbox_out = mod_decrypt(ct, key)
	print(f"Decryption S-Box Output for first round first s-box: {dec2bin(sbox_out[0][0])}")

	text, sbox_out = mod_encrypt(pt, key)
	print(f"Encryption S-Box Output for last round first s-box: {dec2bin(sbox_out[15][0])}")

Decryption S-Box Output for first round first s-box: 0110
Encryption S-Box Output for last round first s-box: 0110
Decryption S-Box Output for first round first s-box: 1001
Encryption S-Box Output for last round first s-box: 1001
Decryption S-Box Output for first round first s-box: 0011
Encryption S-Box Output for last round first s-box: 0011
Decryption S-Box Output for first round first s-box: 0000
Encryption S-Box Output for last round first s-box: 0000
Decryption S-Box Output for first round first s-box: 1001
Encryption S-Box Output for last round first s-box: 1001
Decryption S-Box Output for first round first s-box: 1101
Encryption S-Box Output for last round first s-box: 1101
Decryption S-Box Output for first round first s-box: 1011
Encryption S-Box Output for last round first s-box: 1011
Decryption S-Box Output for first round first s-box: 0010
Encryption S-Box Output for last round first s-box: 0010
Decryption S-Box Output for first round first s-box: 1001
Encryption S-Box Outpu

In [ ]:
# Test on entire dataset
results = []
for ct in ciphertexts:
	text, sbox_out = mod_decrypt(ct, key)
	decrypt_out = int(dec2bin(sbox_out[0][0])[0])
	select_out = dpa_select_function(ct, 60) # correct key is 60 from the paper
	
	if decrypt_out == select_out: 
		results.append(True)
	else:
		print(decrypt_out, select_out)
		results.append(False)
		
print(all(results))

True
